In [1]:
%reload_ext Cython

In [2]:
import cProfile
import tensorflow as tf
import os
import re
import chess
import test
import boardclass
import importlib
import numpy as np
import pickle
import training
import chess.svg
from IPython.display import SVG,display,clear_output
importlib.reload(boardclass)
importlib.reload(test)
importlib.reload(training)
#takes a long time to load, so only run once
data_size = 1# Amount of games to use as trainingdata. max is 2.000.000 (If you want a really good model, you're gonna wait for it :P)

In [3]:
#This cell generates training data
#Outcommented because training data already exists. If you want to generate more or less (tweak data_size), add the code and run this cell

#train_data = training.TrainingData_PQR("../extracted/good_games.txt",data_size)
#bin_file = open("pickled_train_data.bin",mode='wb')
#pickle.dump(train_data,bin_file)
#bin_file.close()

In [4]:
#Loads saved training data

bin_file = open("../pickled_train_data.bin",mode='rb')
train_data = pickle.load(bin_file)
bin_file.close()

FileNotFoundError: [Errno 2] No such file or directory: '../pickled_train_data.bin'

In [ ]:
len(train_data.p)

In [ ]:
#After running the other cells, run this cell to train the model and start the game

render_size = 300
depth = 0
model_path = "model_final.meta"

def make_imported_model(path):
    model = training.Model()
    model.restore(path)
    return model
    
def make_trained_model(train_data):   
    model = training.Model()
    model.run_session(train_data)
    return model
    
#def make_trained_board(player,depth):
#    model = training.Model()
#    model.run_session(train_data,depth)
#    model.save_
##    importlib.reload(training)
#    board = boardclass.ChessBoard(player)#,path="model_final.meta")
#    board.add_model(model)
##    board.train_model(data_size,train_data,depth)
##    board.restore_model()
#    return board

def start_game_with_GUI(player,model,machine_against_machine=False):
    board = boardclass.ChessBoard()
    board.add_model(model)
    display(SVG(chess.svg.board(board=board.python_board,size=render_size)))
    stop = False
    p = re.compile("^[a-h][0-8][a-h][0-8](q|n|r|b)?$")

    while True:
        move = input()
        if move == "exit":
            break
        if board.player == player and not machine_against_machine:
            if p.match(move) == None:
                print("Move: {} is not valid, try again".format(move))
            else:
                board.make_move(move)
                
        else:
            
            score,move = board.return_best_move(depth)
            board.make_move(move)
        clear_output()
        display(SVG(chess.svg.board(board=board.python_board,size=render_size)))
        if board.player_in_chess != None:
            print(board.player_in_chess)
            print("Player {} lost".format(board.player))
            break
model_trained = make_trained_model(train_data)
model_trained.save("model/model_final.ckpt")
model_imported = make_imported_model("model/model_final.ckpt")
model_trained.delete_model_files("model","model_final")
start_game_with_GUI(-1,model_imported,machine_against_machine=False)

In [ ]:
b = chess.Board()
display(SVG(chess.svg.board(board=b,size=render_size)))
while True:
    do = input()
    p = re.compile("^[a-h][0-8][a-h][0-8](q|n|r|b)?$")
    if not p.match(do):
        continue
    clear_output()
    move = chess.Move.from_uci(do)
    b.push(move)
    display(SVG(chess.svg.board(board=b,size=render_size)))   
    

In [ ]:
x_1,y_1,x_2,y_2 = boardlib.algebraic_to_arr_indices("b7a6")
:q
    
print(x_1,y_1,x_2,y_2)
#b = chess.Board()
#square = chess.square(0,0)
#square2 = chess.square(0,2)
#piece = chess.Piece(b.piece_at(square).piece_type,True)
#b.set_piece_at(square2,piece)
#b.push(chess.Move.from_uci("a7a6"))
#b.piece_at(chess.square(0,5))
#b

In [ ]:
def fun(a,b):
    return a+b

s = fun
s(3,5)